In [29]:
from datetime import datetime
startTime = datetime.now()

from itertools import permutations

filepath = 'input.txt'

INSTRUCTION_JUMPS = {'01': 4, '02': 4, '03': 2, '04': 2, '05': 3, '06': 3, '07': 4, '08': 4}
MODES = {'0': lambda i, n, data: data[data[i+n]], '1': lambda i, n, data: data[i+n]}

def run_program(data, input_vals, instruction_pointer = 0, input_pointer = 0):
    i, num_input = instruction_pointer, input_pointer
    while data[i] != 99:
        opcode = str(data[i])[-2:].zfill(2)
        mode = str(data[i])[:-2].zfill(3)
        
        if opcode in ['01', '02', '04', '05', '06', '07', '08']:
            p1 = MODES[mode[2]](i, 1, data)                 
        if opcode in ['01', '02', '05', '06', '07', '08']:
            p2 = MODES[mode[1]](i, 2, data)                                     
        
        if opcode == '01':
            data[data[i+3]] = p1 + p2
        elif opcode == '02':
            data[data[i+3]] = p1 * p2
        elif opcode == '03':
            data[data[i+1]] = input_vals[num_input]
            num_input += 1
        elif opcode == '04':
            return data, i+2, num_input, p1
        elif opcode == '05' and p1 != 0:
            i = p2
            continue
        elif opcode == '06' and p1 == 0:
            i = p2
            continue 
        elif opcode == '07':
            data[data[i+3]] = int(p1 < p2) 
        elif opcode == '08':
            data[data[i+3]] = int(p1 == p2)
        else:
            pass
            
        i += INSTRUCTION_JUMPS[opcode]
        
    return None

def p1_run_amplifiers(a_phase, b_phase, c_phase, d_phase, e_phase):
    a_in = 0
    _, _, _, b_in = run_program(data[:], [a_phase, a_in])
    _, _, _, c_in = run_program(data[:], [b_phase, b_in])
    _, _, _, d_in = run_program(data[:], [c_phase, c_in])
    _, _, _, e_in = run_program(data[:], [d_phase, d_in])
    _, _, _, a_in = run_program(data[:], [e_phase, e_in])
    return a_in
        
def p2_run_amplifiers(a_phase, b_phase, c_phase, d_phase, e_phase):
    a_ins, b_ins, c_ins, d_ins, e_ins = [a_phase, 0], [b_phase], [c_phase], [d_phase], [e_phase]
    instruct_pointer, ins_pointer = [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]
    a_data, b_data, c_data, d_data, e_data = data[:], data[:], data[:], data[:], data[:]

    while True:
        try:
            a_data, instruct_pointer[0], ins_pointer[0], b_in = run_program(a_data, a_ins, instruct_pointer[0], ins_pointer[0])
            b_ins.append(b_in)
            b_data, instruct_pointer[1], ins_pointer[1], c_in = run_program(b_data, b_ins, instruct_pointer[1], ins_pointer[1])
            c_ins.append(c_in)
            c_data, instruct_pointer[2], ins_pointer[2], d_in = run_program(c_data, c_ins, instruct_pointer[2], ins_pointer[2])
            d_ins.append(d_in)
            d_data, instruct_pointer[3], ins_pointer[3], e_in = run_program(d_data, d_ins, instruct_pointer[3], ins_pointer[3])
            e_ins.append(e_in)
            e_data, instruct_pointer[4], ins_pointer[4], a_in = run_program(e_data, e_ins, instruct_pointer[4], ins_pointer[4])
            a_ins.append(a_in)
        
        except TypeError:
            return a_ins[-1]
        
def check_phase_sequences(min_phase, max_phase, run_script):
    phase_sequences = permutations(range(min_phase, max_phase))               
    max_thruster = 0
    for a, b, c, d, e in phase_sequences:
        thruster = run_script(a, b, c, d, e)

        if thruster > max_thruster:
            max_thruster = thruster

    return max_thruster

data = []
with open(filepath, 'r') as f:
    for line in f.readlines():
        data = [int(i) for i in line.strip().split(',')]
f.close()

print(check_phase_sequences(0, 5, p1_run_amplifiers))
print(check_phase_sequences(5, 10, p2_run_amplifiers))
print(str(datetime.now() - startTime)[:-3])

914828
17956613
0:00:00.070
